# **Project 2: Publishing Assessments**

In [1]:
import json
from pyspark.sql import Row

## **0. Linux Commands and Instructions for Project**

#### 1) Command to Bring Up the Cluster 
    docker-compose up -d

#### 2) Command to Create the Kafka Topic Assessments 
    docker-compose exec kafka kafka-topics --create --topic assessments --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181

#### 3) Command to Publish the Assessments Json Data to the Kafka Topic Using Kafkacat
    docker-compose exec mids bash -c "cat /w205/project-2-hfarb/assessment-attempts-20180128-121051-nested.json | jq '.[]' -c | kafkacat -P -b kafka:29092 -t assessments”

#### 4) Create a symbolic link in the Spark container to the /205 mount point
     docker-compose exec spark bash
     ln -s /w205 w205
     exit
#### 5) Run an Enhanced Version of the Pyspark Command Line to Target Jupyter Notebook
     docker-compose exec spark env PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port 8888 --ip 0.0.0.0 --allow-root' pyspark
#### 6)  Change 0.0.0.0 to the External Ip Address for your Google Cloud Virtual Machine and Open Ingonito Browser.
     Open a new Google Chrome browser incognito window, and copy and paste the URL with the modified ip address from    and the Jupyter Notebook should come up.
#### 7) Command to Shutdown the Cluster
     docker-compose down
     

## **1. Subscribe to the Kafka Topic and Create Data Frames and Temporary Tables**

### 1.1 Create a Data Frame by Subscribing to the Kafka Topic

In [2]:
raw_assessments = spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe","assessments").option("startingOffsets", "earliest").option("endingOffsets", "latest").load() 

In [3]:
raw_assessments.cache()

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [4]:
raw_assessments.show()

+----+--------------------+-----------+---------+------+--------------------+-------------+
| key|               value|      topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----------+---------+------+--------------------+-------------+
|null|[7B 22 6B 65 65 6...|assessments|        0|     0|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     1|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     2|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     3|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     4|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     5|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     6|1969-12-31 23:59:...|            0|
|null|[7B 22 6B 65 65 6...|assessments|        0|     7|1969-12-31 23:59:...|   

### 1.2 Convert the Json Data to a String 

In [5]:
assessments = raw_assessments.select(raw_assessments.value.cast('string'))

In [6]:
assessments.show()

+--------------------+
|               value|
+--------------------+
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
+--------------------+
only showing top 20 rows



### 1.3 Convert the String Json Data into a Data Frame

In [7]:
extracted_assessments = assessments.rdd.map(lambda x: Row(**json.loads(x.value))).toDF()

In [8]:
extracted_assessments.show()

+--------------------+-------------+--------------------+------------------+--------------------+------------------+------------+--------------------+--------------------+--------------------+
|        base_exam_id|certification|           exam_name|   keen_created_at|             keen_id|    keen_timestamp|max_attempts|           sequences|          started_at|        user_exam_id|
+--------------------+-------------+--------------------+------------------+--------------------+------------------+------------+--------------------+--------------------+--------------------+
|37f0a30a-7464-11e...|        false|Normal Forms and ...| 1516717442.735266|5a6745820eb8ab000...| 1516717442.735266|         1.0|Map(questions -> ...|2018-01-23T14:23:...|6d4089e4-bde5-4a2...|
|37f0a30a-7464-11e...|        false|Normal Forms and ...| 1516717377.639827|5a674541ab6b0a000...| 1516717377.639827|         1.0|Map(questions -> ...|2018-01-23T14:21:...|2fec1534-b41f-441...|
|4beeac16-bb83-4d5...|        false

### 1.3.1. Create Temporary Table 'assessments' from Extracted Assessments Data Frame to Allow In Memory Queries

In [9]:
extracted_assessments.registerTempTable('assessments')

## **2. Business Questions and Answers Using Spark SQL Against MPP Dataframe in Memory**

### **2.1 What Are the Top 10 Most Taken Exams and the Top 10 Least Taken?**

I use the temporary table 'assessments' to answer this question.

#### 2.1.1. Most Taken Exams

In [10]:
spark.sql("select exam_name, count(exam_name) as count from assessments group by exam_name order by count desc limit 10").show(20,False)

+-----------------------------------------------------------+-----+
|exam_name                                                  |count|
+-----------------------------------------------------------+-----+
|Learning Git                                               |394  |
|Introduction to Python                                     |162  |
|Introduction to Java 8                                     |158  |
|Intermediate Python Programming                            |158  |
|Learning to Program with R                                 |128  |
|Introduction to Machine Learning                           |119  |
|Software Architecture Fundamentals Understanding the Basics|109  |
|Beginning C# Programming                                   |95   |
|Learning Eclipse                                           |85   |
|Learning Apache Maven                                      |80   |
+-----------------------------------------------------------+-----+



#### 2.1.2. Least Taken Exams

In [11]:
spark.sql("select exam_name, count(exam_name) as count from assessments group by exam_name order by count limit 10").show(truncate = False)

+-------------------------------------------------+-----+
|exam_name                                        |count|
+-------------------------------------------------+-----+
|Nulls, Three-valued Logic and Missing Information|1    |
|Native Web Apps for Android                      |1    |
|Learning to Visualize Data with D3.js            |1    |
|Operating Red Hat Enterprise Linux Servers       |1    |
|The Closed World Assumption                      |2    |
|Arduino Prototyping Techniques                   |2    |
|What's New in JavaScript                         |2    |
|Learning Spring Programming                      |2    |
|Hibernate and JPA Fundamentals                   |2    |
|Understanding the Grails 3 Domain Model          |2    |
+-------------------------------------------------+-----+



### **2.2 What Are the Exams with the Highest Average Scores and the Exams with the Lowest Average Scores?**

In [12]:
def my_lambda_exam_correct_total(x):
    '''Unroll the [sequences][counts] dictionary and check it piece meal to make sure it exists before referencing it.
       Add correct and total values to my_list and also add the relevant exam names.
       '''
    raw_dict = json.loads(x.value)
    my_list = []
    
    if "sequences" in raw_dict:
        
        if "counts" in raw_dict["sequences"]:
            
            if "correct" in raw_dict["sequences"]["counts"] and "total" in raw_dict["sequences"]["counts"]:
                    
                my_dict = {"correct": raw_dict["sequences"]["counts"]["correct"], 
                           "total": raw_dict["sequences"]["counts"]["total"],
                           "exam": raw_dict["exam_name"],}
                my_list.append(Row(**my_dict))
    
    return my_list

In [13]:
# create a data frame from the assessments data frame by applying the my_lambda_exam_correct_total function 
my_exam_correct_total = assessments.rdd.flatMap(my_lambda_exam_correct_total).toDF()

In [14]:
#create temporary table 'cet' to allow in memory queries
my_exam_correct_total.registerTempTable('cet')

In [15]:
#show what the 'cet' temporary table looks like
spark.sql("select * from cet limit 10").show(truncate = False)

+-------+-------------------------------------------+-----+
|correct|exam                                       |total|
+-------+-------------------------------------------+-----+
|2      |Normal Forms and All That Jazz Master Class|4    |
|1      |Normal Forms and All That Jazz Master Class|4    |
|3      |The Principles of Microservices            |4    |
|2      |The Principles of Microservices            |4    |
|3      |Introduction to Big Data                   |4    |
|5      |Learning Git                               |5    |
|1      |Git Fundamentals for Web Developers        |1    |
|5      |Introduction to Python                     |5    |
|4      |Intermediate Python Programming            |4    |
|0      |Introduction to Python                     |5    |
+-------+-------------------------------------------+-----+



#### 2.2.1. Exams with Highest Averages

In [16]:
spark.sql("select exam, avg(correct / total)*100 as avg_score from cet group by exam order by avg_score desc limit 10").show(truncate = False)

+-----------------------------------------------------------------------+-----------------+
|exam                                                                   |avg_score        |
+-----------------------------------------------------------------------+-----------------+
|Nulls, Three-valued Logic and Missing Information                      |100.0            |
|The Closed World Assumption                                            |100.0            |
|Learning to Visualize Data with D3.js                                  |100.0            |
|Learning SQL for Oracle                                                |97.72727272727273|
|Introduction to Java 8                                                 |87.59493670886073|
|Introduction to Amazon Web Services (AWS) - EC2 Deployment Fundamentals|83.33333333333334|
|Introduction to Apache Spark                                           |83.33333333333334|
|Cloud Native Architecture Fundamentals                                 |80.0000

#### 2.2.2. Exams with Lowest Averages

In [17]:
spark.sql("select exam, avg(correct / total)*100 as avg_score from cet group by exam order by avg_score limit 10").show(truncate = False)

+-------------------------------------------+------------------+
|exam                                       |avg_score         |
+-------------------------------------------+------------------+
|Client-Side Data Storage for Web Developers|20.0              |
|View Updating                              |25.0              |
|Native Web Apps for Android                |25.0              |
|Arduino Prototyping Techniques             |33.33333333333333 |
|Mastering Advanced Git                     |36.029411764705884|
|Nullology                                  |37.5              |
|Mastering Web Views                        |41.66666666666667 |
|Building Web Services with Java            |41.66666666666667 |
|Web & Native Working Together              |41.66666666666667 |
|Cloud Computing With AWS                   |42.64705882352941 |
+-------------------------------------------+------------------+



### ** 2.3. What are the Exams with the Most Questions?**

In [18]:
spark.sql("select distinct(exam), total as number_questions from cet order by number_questions desc limit 10").show(truncate = False)

+------------------------------------------+----------------+
|exam                                      |number_questions|
+------------------------------------------+----------------+
|Operating Red Hat Enterprise Linux Servers|20              |
|Great Bash                                |10              |
|Learning Linux System Administration      |8               |
|Learning to Program with R                |7               |
|Introduction to Data Science with R       |7               |
|Being a Better Introvert                  |7               |
|Understanding the Grails 3 Domain Model   |7               |
|What's New in JavaScript                  |7               |
|Arduino Inputs                            |6               |
|Arduino Prototyping Techniques            |6               |
+------------------------------------------+----------------+



## **3. Write to HDFS in Parquet Format for each Data Frame Created**

In [19]:
raw_assessments.write.mode("overwrite").parquet("/tmp/raw_assessments")
assessments.write.mode("overwrite").parquet("/tmp/assessments")
extracted_assessments.write.mode("overwrite").parquet("/tmp/extracted_assessments")
my_exam_correct_total.write.mode("overwrite").parquet("/tmp/my_exam_correct_total")